# Process and prepare the raw data for use in the lesson

The raw gravity and topography data are generated from [spherical harmonic models](https://en.wikipedia.org/wiki/Spherical_harmonics) using the [ICGEM Calculation Service](http://icgem.gfz-potsdam.de). The data are distributed in text files and heights are defined relative to the geoid (orthometric heights). We need to convert the obeservation heights and topography to geometric heights (relative to the ellipsoid) using the geoid heights also downloaded from ICGEM. We'll also save this data in [netCDF](https://www.unidata.ucar.edu/software/netcdf/) files for easier loading with [xarray](http://xarray.pydata.org/).

The model used to generate the gravity and geoid height data is EIGEN-6c4. The topography data is interpolated from the ETOPO1 model.

## Import the required libraries

We need the following libraries to do this work.

In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
# Load a custom function from icgem.py to load the raw data
from icgem import load_icgem_gdf

## Loading ICGEM files

The following function loads the data from the ICGEM text files along with some metadata from the headers.

In [4]:
360 - 210

150

In [ ]:
grav_file = 'data/eigen-6c3stat-havai.gdf'
topo_file = 'data/e